# DeepFabric ile Sentetik Veri Üretimi - Kapsamlı Rehber

Bu notebook, DeepFabric kütüphanesini kullanarak lokal LLM modelleri ile sentetik veri üretimini adım adım göstermektedir.

## DeepFabric Nedir?

DeepFabric, yapay zeka modelleri için yüksek kaliteli eğitim verileri oluşturmak üzere tasarlanmış bir sentetik veri üretim çerçevesidir. NVIDIA, Google ve GitHub mühendisleri tarafından geliştirilmiştir.

### Temel Özellikler:

- **Hiyerarşik Konu Ağacı**: Bir ana konudan alt konular oluşturur ve bunlardan sentetik veriler üretir
- **Çoklu LLM Desteği**: OpenAI, Anthropic, Google, Ollama ve OpenAI-uyumlu API'ler
- **Farklı Veri Tipleri**: Soru-cevap, konuşmalar, araç çağrıları, akıl yürütme zincirleri
- **Esnek Formatlar**: TRL, ChatML, Alpaca, özel formatlar
- **Lokal ve Bulut**: Hem lokal hem de bulut tabanlı modeller

### Nasıl Çalışır?

1. **Konu Ağacı Oluşturma**: Verdiğiniz ana konudan hiyerarşik alt konular üretir
2. **Veri Üretimi**: Her konu için soru-cevap çiftleri, konuşmalar veya örnekler oluşturur
3. **Formatlama**: Üretilen veriyi model eğitimi için uygun formata dönüştürür

---

## 1. Kurulum

DeepFabric'i pip ile kuruyoruz:

In [ ]:
# DeepFabric kütüphanesini kur
!pip install deepfabric -q

In [ ]:
# Gerekli kütüphaneleri import et
import os
import json
from pathlib import Path

## 2. Lokal LLM API Konfigürasyonu

### Seçenek 1: OpenAI-Uyumlu API (vLLM, text-generation-webui, LM Studio vb.)

Eğer lokal Llama modelinizi OpenAI-uyumlu bir API ile çalıştırıyorsanız (örneğin vLLM, text-generation-webui, LM Studio):

```python
# API bilgilerinizi buraya girin
os.environ['OPENAI_API_KEY'] = 'dummy-key'  # Lokal API için genelde gerekli değil ama zorunlu
os.environ['OPENAI_BASE_URL'] = 'http://localhost:8000/v1'  # API URL'iniz
```

### Seçenek 2: Ollama ile

Eğer Ollama kullanıyorsanız:
```bash
# Terminal'de Ollama'yı başlatın
ollama serve
```

## 2.6 Watsonx Kullanımı - Özet ve Karşılaştırma

### 🎯 İki Yöntem:

#### **Yöntem 1: LiteLLM Proxy + DeepFabric** (Önerilen)

**Avantajlar:**
- ✅ DeepFabric'in tüm özelliklerini kullanabilirsiniz (konu ağacı, otomatik format dönüşümü vb.)
- ✅ YAML konfigürasyonu ile detaylı kontrol
- ✅ Birden fazla LLM provider'ı tek bir sistemde kullanabilirsiniz

**Dezavantajlar:**
- ❌ LiteLLM proxy kurulumu gerekir
- ❌ Ekstra bir servis çalıştırmanız gerekir

**Ne zaman kullanmalı:**
- Büyük ölçekli veri üretimi yapacaksanız
- Farklı formatlar (TRL, Alpaca, ChatML vb.) gerekiyorsa
- Konu ağacı (topic tree) özelliğini kullanmak istiyorsanız

---

#### **Yöntem 2: Direkt Watsonx SDK**

**Avantajlar:**
- ✅ Ekstra servis gerekmez, doğrudan Watsonx API kullanılır
- ✅ Basit ve hızlı kurulum
- ✅ Tam kontrol (özel prompt formatları, parsing vb.)

**Dezavantajlar:**
- ❌ DeepFabric özellikleri yok
- ❌ Kendi veri üretim logic'inizi yazmanız gerekir
- ❌ Format dönüşümleri manuel

**Ne zaman kullanmalı:**
- Küçük ölçekli veri üretimi için (< 100 örnek)
- Hızlı prototipleme için
- Özel veri formatı gerekiyorsa

---

### 📝 Hangi Yöntemi Seçmeli?

| Kriter | LiteLLM + DeepFabric | Direkt Watsonx SDK |
|--------|---------------------|-------------------|
| Kurulum Kolaylığı | ⭐⭐⭐ | ⭐⭐⭐⭐⭐ |
| Özellik Zenginliği | ⭐⭐⭐⭐⭐ | ⭐⭐ |
| Ölçeklenebilirlik | ⭐⭐⭐⭐⭐ | ⭐⭐⭐ |
| Esneklik | ⭐⭐⭐⭐ | ⭐⭐⭐⭐⭐ |

**Öneri:** Eğer DeepFabric'i öğrenmek ve uzun vadeli kullanmak istiyorsanız **Yöntem 1**'i seçin. Hızlı test için **Yöntem 2** yeterlidir.

In [ ]:
# Direkt Watsonx ile üretilen veriyi incele
if os.path.exists('dataset_direct_watsonx.jsonl'):
    print("\n📊 DİREKT WATSONX SDK İLE ÜRETİLEN VERİ:\n")
    print("=" * 80)
    
    with open('dataset_direct_watsonx.jsonl', 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            data = json.loads(line)
            print(f"\n🔹 Soru-Cevap {i+1}:")
            print("-" * 80)
            
            for msg in data['messages']:
                role = msg['role'].upper()
                content = msg['content']
                print(f"\n[{role}]")
                print(content)
            
            print("\n" + "=" * 80)
else:
    print("❌ Veri seti bulunamadı. Önce yukarıdaki cell'i çalıştırın.")

In [ ]:
# Direkt Watsonx SDK ile txt dosyasından veri üret
# Önce txt dosyasını okuyalım (eğer yoksa oluşturulacak)

if not os.path.exists('ornek_konu.txt'):
    # Örnek txt oluştur
    ornek_icerik = """
Yapay Zeka ve Etik

Yapay zeka sistemlerinin gelişimi beraberinde önemli etik soruları getirmektedir:

1. Önyargı ve Adalet: AI sistemleri eğitim verilerindeki önyargıları yansıtabilir
2. Gizlilik: Kişisel verilerin korunması kritik önem taşır
3. Şeffaflık: AI kararlarının açıklanabilir olması gerekir
4. Sorumluluk: AI hatalarından kim sorumludur?
5. İş Gücü Etkisi: Otomasyon ve istihdam dengesi

Bu konular yapay zeka geliştirirken dikkate alınmalıdır.
"""
    with open('ornek_konu.txt', 'w', encoding='utf-8') as f:
        f.write(ornek_icerik)
    print("✓ Örnek txt dosyası oluşturuldu")

# Txt dosyasını oku
with open('ornek_konu.txt', 'r', encoding='utf-8') as f:
    konu = f.read()

print(f"📖 Konu metni okundu ({len(konu)} karakter)")
print("\nŞimdi Watsonx ile sentetik veri üretiliyor...")
print("(Bu işlem birkaç dakika sürebilir)\n")

# Watsonx ile veri üret
# NOT: Bu cell'i çalıştırmak için Watsonx API bilgilerinizin doğru olması gerekir
try:
    dataset = watsonx_sentetik_veri_uret(
        konu_metni=konu,
        soru_sayisi=5  # Test için 5 soru-cevap
    )
    
    # JSONL formatında kaydet
    output_file = 'dataset_direct_watsonx.jsonl'
    with open(output_file, 'w', encoding='utf-8') as f:
        for item in dataset:
            f.write(json.dumps(item, ensure_ascii=False) + '\n')
    
    print(f"\n💾 Veri seti kaydedildi: {output_file}")
    
except Exception as e:
    print(f"\n❌ Hata: {e}")
    print("\nLütfen Watsonx API bilgilerinizi kontrol edin (WATSONX_API_KEY, WATSONX_PROJECT_ID, vb.)")

In [ ]:
# Direkt Watsonx SDK ile Sentetik Veri Üretimi

from ibm_watsonx_ai.foundation_models import Model
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
import json
import time

def watsonx_sentetik_veri_uret(
    konu_metni,
    soru_sayisi=10,
    api_key=None,
    project_id=None,
    url=None,
    model_id=None
):
    """
    Watsonx kullanarak verilen konu metninden soru-cevap çiftleri üretir.
    
    Args:
        konu_metni: Ana konu metni (txt dosyasından okunmuş içerik)
        soru_sayisi: Kaç soru-cevap çifti üretileceği
        api_key: Watsonx API key
        project_id: Watsonx project ID
        url: Watsonx URL
        model_id: Kullanılacak model ID
    
    Returns:
        List of dict: Soru-cevap çiftleri
    """
    
    # Watsonx model konfigürasyonu
    model_params = {
        GenParams.DECODING_METHOD: "greedy",
        GenParams.MAX_NEW_TOKENS: 500,
        GenParams.TEMPERATURE: 0.7,
        GenParams.MIN_NEW_TOKENS: 50,
    }
    
    credentials = {
        "url": url or WATSONX_URL,
        "apikey": api_key or WATSONX_API_KEY
    }
    
    # Model instance
    model = Model(
        model_id=model_id or WATSONX_MODEL,
        params=model_params,
        credentials=credentials,
        project_id=project_id or WATSONX_PROJECT_ID
    )
    
    # Veri seti
    dataset = []
    
    print(f"🚀 {soru_sayisi} soru-cevap çifti üretiliyor...\n")
    
    for i in range(soru_sayisi):
        # Prompt: Konu metninden soru-cevap üret
        prompt = f"""Aşağıdaki konu hakkında 1 adet eğitici soru ve detaylı cevap üret.

Konu:
{konu_metni}

Lütfen şu formatta yanıt ver:
SORU: [soru metni]
CEVAP: [detaylı cevap metni]

Soru numarası: {i+1}/{soru_sayisi}
"""
        
        try:
            # Model'den yanıt al
            yanit = model.generate_text(prompt=prompt)
            
            # Soru ve cevabı parse et
            if "SORU:" in yanit and "CEVAP:" in yanit:
                parts = yanit.split("CEVAP:")
                soru = parts[0].replace("SORU:", "").strip()
                cevap = parts[1].strip()
                
                # ChatML formatında kaydet
                veri_ornegi = {
                    "messages": [
                        {"role": "system", "content": "Sen yardımcı bir Türkçe AI asistanısın."},
                        {"role": "user", "content": soru},
                        {"role": "assistant", "content": cevap}
                    ]
                }
                
                dataset.append(veri_ornegi)
                print(f"✓ Örnek {i+1} oluşturuldu")
            else:
                print(f"⚠ Örnek {i+1} formatı hatalı, atlandı")
            
            # Rate limiting için kısa bekleme
            time.sleep(0.5)
            
        except Exception as e:
            print(f"❌ Örnek {i+1} hatası: {e}")
            continue
    
    print(f"\n✅ Toplam {len(dataset)} soru-cevap çifti oluşturuldu!")
    return dataset

print("✓ Fonksiyon tanımlandı: watsonx_sentetik_veri_uret()")

## 2.5 Alternatif: LiteLLM Olmadan Direkt Watsonx Kullanımı

Eğer LiteLLM proxy kurmak istemiyorsanız, Watsonx SDK'sını doğrudan kullanarak basit bir sentetik veri üretimi yapabilirsiniz.

Bu yöntemde DeepFabric kullanmayacağız, ancak benzer mantıkla kendi sentetik veri üretim script'imizi yazacağız.

In [ ]:
# Watsonx ile üretilen soru-cevapları incele
if os.path.exists('dataset_watsonx_from_txt.jsonl'):
    print("\n📊 WATSONX İLE ÜRETİLEN SORU-CEVAPLAR:\n")
    print("=" * 80)
    
    with open('dataset_watsonx_from_txt.jsonl', 'r', encoding='utf-8') as f:
        veri_sayisi = 0
        for i, line in enumerate(f):
            veri_sayisi += 1
            if i < 3:  # İlk 3 örneği göster
                data = json.loads(line)
                print(f"\n🔹 Örnek {i+1}:")
                print("-" * 80)
                
                if 'messages' in data:
                    for msg in data['messages']:
                        role = msg.get('role', 'unknown').upper()
                        content = msg.get('content', '')
                        print(f"\n[{role}]")
                        # Türkçe karakterleri doğru göster
                        if len(content) <= 500:
                            print(content)
                        else:
                            print(content[:500] + "...")
                else:
                    print(json.dumps(data, indent=2, ensure_ascii=False)[:1000])
                
                print("\n" + "=" * 80)
    
    print(f"\n📈 Toplam {veri_sayisi} soru-cevap çifti üretildi!")
    
else:
    print("❌ Veri seti henüz oluşturulmadı.")
    print("Önce yukarıdaki cell'i çalıştırın.")

In [ ]:
# Watsonx ile txt dosyasından soru-cevap üret
!deepfabric generate \
  --config watsonx_txt_config.yaml \
  --dataset-save-as dataset_watsonx_from_txt.jsonl

print("\n✅ Watsonx ile txt dosyasından sentetik veri üretimi tamamlandı!")
print("Sonuçlar: dataset_watsonx_from_txt.jsonl")

In [ ]:
# Watsonx ile txt içeriğinden soru-cevap üretimi için YAML config oluştur
watsonx_txt_config = f"""
dataset_system_prompt: |
  Sen yardımcı bir Türkçe AI asistanısın. Kullanıcılara verilen konular 
  hakkında detaylı ve anlaşılır bilgiler veriyorsun.

topic_tree:
  # Txt dosyasının içeriğini konu olarak kullanıyoruz
  topic_prompt: |
    Aşağıdaki içerik hakkında detaylı alt konular ve soru-cevap çiftleri oluştur:
    
    {txt_icerik}
  
  provider: "{PROVIDER}"
  model: "{MODEL_NAME}"
  temperature: 0.7
  degree: 3  # Her konudan 3 alt konu
  depth: 2   # 2 seviye derinlik

data_engine:
  generation_system_prompt: |
    Verilen konular hakkında Türkçe soru-cevap çiftleri oluştur.
    Her cevap:
    - Net ve anlaşılır olmalı
    - Örnekler içermeli
    - Eğitici olmalı
    - Orta seviye teknik detay içermeli
  
  temperature: 0.6
  provider: "{PROVIDER}"
  model: "{MODEL_NAME}"

dataset:
  creation:
    num_steps: 20      # 20 soru-cevap çifti üret
    batch_size: 2      # 2'şer 2'şer üret
    sys_msg: true
  template: "builtin://chatml.py"
"""

# Config dosyasını kaydet
with open('watsonx_txt_config.yaml', 'w', encoding='utf-8') as f:
    f.write(watsonx_txt_config)

print("✓ Watsonx txt-based konfigürasyon oluşturuldu: watsonx_txt_config.yaml")

In [ ]:
# Kendi txt dosyanızı okuyun (örnek olarak ornek_konu.txt kullanıyoruz)
# Eğer kendi dosyanızı kullanmak isterseniz dosya yolunu değiştirin

TXT_DOSYA_YOLU = "ornek_konu.txt"  # Kendi dosyanızın yolunu girin

# Txt dosyasını oku
try:
    with open(TXT_DOSYA_YOLU, 'r', encoding='utf-8') as f:
        txt_icerik = f.read()
    
    print(f"✓ Dosya okundu: {TXT_DOSYA_YOLU}")
    print(f"  İçerik uzunluğu: {len(txt_icerik)} karakter")
    print(f"\n--- Dosya İçeriği (İlk 300 karakter) ---")
    print(txt_icerik[:300])
    print("...")
    
except FileNotFoundError:
    print(f"❌ Dosya bulunamadı: {TXT_DOSYA_YOLU}")
    print("Önce bir txt dosyası oluşturun veya mevcut dosyanın yolunu kontrol edin.")

## 2.4 Watsonx ile TXT Dosyasından Soru-Cevap Üretimi

Kendi txt dosyanızı kullanarak Watsonx ile sentetik soru-cevap setleri oluşturabilirsiniz.

In [ ]:
# Watsonx ile basit sentetik veri üretimi
# NOT: LiteLLM proxy'nin çalıştığından emin olun!

!deepfabric generate \
  --mode tree \
  --provider {PROVIDER} \
  --model {MODEL_NAME} \
  --depth 2 \
  --degree 2 \
  --num-steps 10 \
  --topic-prompt "Yapay Zeka ve Makine Öğrenmesi Temelleri" \
  --dataset-save-as dataset_watsonx.jsonl

print("\n✓ Watsonx ile veri üretimi tamamlandı!")

## 2.3 Watsonx ile Sentetik Veri Üretimi - Örnek

Artık Watsonx modelinizi kullanarak sentetik veri üretebilirsiniz!

In [ ]:
# LiteLLM Proxy ile DeepFabric Kullanımı
# ÖNCE: Terminal'de 'litellm --config litellm_config.yaml --port 8000' komutunu çalıştırın

# OpenAI API konfigürasyonunu LiteLLM proxy'ye yönlendir
os.environ['OPENAI_API_KEY'] = 'sk-1234'  # LiteLLM master_key
os.environ['OPENAI_BASE_URL'] = 'http://localhost:8000'  # LiteLLM proxy URL

PROVIDER = 'openai'
MODEL_NAME = 'watsonx-llama'  # LiteLLM config'deki model_name

print("✓ DeepFabric, Watsonx'e LiteLLM proxy üzerinden bağlanacak")
print(f"  Provider: {PROVIDER}")
print(f"  Model: {MODEL_NAME}")
print(f"  Proxy URL: {os.environ['OPENAI_BASE_URL']}")

In [ ]:
# LiteLLM Proxy Konfigürasyon Dosyası Oluştur
litellm_config = f"""
model_list:
  - model_name: watsonx-llama
    litellm_params:
      model: watsonx/{WATSONX_MODEL}
      api_key: {WATSONX_API_KEY}
      project_id: {WATSONX_PROJECT_ID}
      url: {WATSONX_URL}

general_settings:
  master_key: sk-1234  # Basit bir key, güvenlik için değiştirin
"""

# Config dosyasını kaydet
with open('litellm_config.yaml', 'w') as f:
    f.write(litellm_config)

print("✓ LiteLLM config dosyası oluşturuldu: litellm_config.yaml")
print("\nŞimdi yeni bir terminal açıp şu komutu çalıştırın:")
print("  litellm --config litellm_config.yaml --port 8000")
print("\nProxy başladıktan sonra DeepFabric'i şu şekilde kullanabilirsiniz:")

## 2.2 Watsonx ile DeepFabric Kullanımı

DeepFabric doğrudan Watsonx'i desteklemez, ancak **LiteLLM proxy** kullanarak Watsonx'i OpenAI-uyumlu bir endpoint haline getirebiliriz.

### Yöntem 1: LiteLLM Proxy Kullanımı (Önerilen)

LiteLLM, Watsonx dahil birçok LLM provider'ı OpenAI API formatına dönüştüren bir proxy'dir.

**Adımlar:**

1. LiteLLM proxy config dosyası oluştur
2. Proxy'yi başlat (localhost:8000)
3. DeepFabric'i bu proxy ile kullan

### Yöntem 2: Watsonx API Wrapper ile Özel Implementation

Watsonx API'sini doğrudan kullanarak veri üretimi yapabiliriz (DeepFabric olmadan, benzer yaklaşımla).

In [ ]:
# Watsonx API bağlantısını test edelim
from ibm_watsonx_ai.foundation_models import Model
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams

# Watsonx model parametreleri
model_params = {
    GenParams.DECODING_METHOD: "greedy",
    GenParams.MAX_NEW_TOKENS: 100,
    GenParams.TEMPERATURE: 0.7,
}

# Watsonx credentials
credentials = {
    "url": WATSONX_URL,
    "apikey": WATSONX_API_KEY
}

try:
    # Model instance oluştur
    watsonx_model = Model(
        model_id=WATSONX_MODEL,
        params=model_params,
        credentials=credentials,
        project_id=WATSONX_PROJECT_ID
    )
    
    # Test prompt
    test_prompt = "Python programlama dilinin en önemli özelliği nedir?"
    response = watsonx_model.generate_text(prompt=test_prompt)
    
    print("✅ Watsonx API bağlantısı başarılı!")
    print(f"\nTest Prompt: {test_prompt}")
    print(f"\nModel Yanıtı:\n{response}")
    
except Exception as e:
    print(f"❌ Bağlantı hatası: {e}")
    print("\nLütfen API key, Project ID ve URL bilgilerinizi kontrol edin.")

In [ ]:
# WATSONX API KONFİGÜRASYONU
# Kendi bilgilerinizi buraya girin

# 1. IBM Cloud API bilgileriniz
WATSONX_API_KEY = "your-ibm-cloud-api-key-here"  # IBM Cloud IAM API Key
WATSONX_PROJECT_ID = "your-project-id-here"      # Watsonx Project ID
WATSONX_URL = "https://us-south.ml.cloud.ibm.com"  # Region'ınıza göre değişebilir

# 2. Model seçimi
# Watsonx'te kullanabileceğiniz modeller:
# - "meta-llama/llama-3-70b-instruct"
# - "meta-llama/llama-3-8b-instruct"
# - "ibm/granite-13b-chat-v2"
# - "mistralai/mixtral-8x7b-instruct-v01"
WATSONX_MODEL = "meta-llama/llama-3-70b-instruct"  # Kendi modelinizi seçin

# 3. Watsonx credentials'ları environment variable olarak ayarla
os.environ['WATSONX_APIKEY'] = WATSONX_API_KEY
os.environ['WATSONX_PROJECT_ID'] = WATSONX_PROJECT_ID
os.environ['WATSONX_URL'] = WATSONX_URL

print("✓ Watsonx API Konfigürasyonu tamamlandı")
print(f"  Model: {WATSONX_MODEL}")
print(f"  URL: {WATSONX_URL}")
print(f"  Project ID: {WATSONX_PROJECT_ID[:20]}...")  # İlk 20 karakter

In [ ]:
# Watsonx entegrasyonu için gerekli kütüphaneleri kur
# Seçenek 1: LiteLLM ile (Önerilen)
!pip install litellm ibm-watsonx-ai -q

# Seçenek 2: Sadece Watsonx SDK
# !pip install ibm-watsonx-ai -q

print("✓ Watsonx kütüphaneleri kuruldu")

## 2.1 Watsonx Platform Konfigürasyonu (IBM Cloud)

**Watsonx**, IBM Cloud platformunda çalışan güçlü bir LLM servisidir. Llama, Granite ve diğer açık kaynak modelleri API üzerinden kullanmanıza olanak tanır.

### Watsonx API Bilgilerinizi Alma:

1. **IBM Cloud Console**'a giriş yapın: https://cloud.ibm.com
2. **Watsonx.ai** servisinize gidin
3. Aşağıdaki bilgileri alın:
   - **API Key**: IBM Cloud IAM API anahtarınız
   - **Project ID**: Watsonx projenizin ID'si
   - **URL**: Watsonx API endpoint URL'i (genelde `https://us-south.ml.cloud.ibm.com` veya region'ınıza göre değişir)

### Watsonx API'nin DeepFabric ile Kullanımı:

Watsonx API'si OpenAI-uyumlu **değildir**, ancak Python ile direkt API çağrıları yaparak veya bir wrapper kullanarak DeepFabric'e entegre edebiliriz.

**İki Seçenek:**

1. **LiteLLM Kullanarak** (Önerilen - Watsonx'i OpenAI-compatible hale getirir)
2. **Direkt Watsonx SDK** ile özel implementation

Aşağıda her iki yöntemi göreceğiz.

In [ ]:
# API KONFİGÜRASYONU - KENDİ BİLGİLERİNİZİ GİRİN

# Seçenek 1: OpenAI-uyumlu lokal API
os.environ['OPENAI_API_KEY'] = 'sk-dummy-key'  # Lokal için dummy key
os.environ['OPENAI_BASE_URL'] = 'http://localhost:8000/v1'  # Kendi API URL'iniz

# Model adı - API'nizde kullandığınız model adı
LOCAL_MODEL_NAME = 'llama-3.1-8b'  # Kendi model adınızı girin

# Eğer Ollama kullanıyorsanız:
# PROVIDER = 'ollama'
# LOCAL_MODEL_NAME = 'llama3.1:8b'

PROVIDER = 'openai'  # OpenAI-uyumlu API için 'openai', Ollama için 'ollama'

print(f"✓ API Konfigürasyonu tamamlandı")
print(f"  Provider: {PROVIDER}")
print(f"  Model: {LOCAL_MODEL_NAME}")
print(f"  Base URL: {os.environ.get('OPENAI_BASE_URL', 'default')}")

## 3. Basit Örnek: CLI ile Sentetik Veri Üretimi

DeepFabric'in en basit kullanımı CLI (komut satırı) üzerinden yapılır:

In [ ]:
# Basit bir sentetik veri seti oluştur
# Bu komut:
# 1. "Python Programlama" konusundan konu ağacı oluşturur (depth=2, degree=2)
# 2. Her konu için 8 veri örneği üretir (num-steps=8)
# 3. Sonuçları 'dataset_basit.jsonl' dosyasına kaydeder

!deepfabric generate \
  --mode tree \
  --provider {PROVIDER} \
  --model {LOCAL_MODEL_NAME} \
  --depth 2 \
  --degree 2 \
  --num-steps 8 \
  --topic-prompt "Python Programlama Temelleri" \
  --dataset-save-as dataset_basit.jsonl

In [ ]:
# Oluşturulan veriyi inceleyelim
if os.path.exists('dataset_basit.jsonl'):
    with open('dataset_basit.jsonl', 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            if i < 2:  # İlk 2 örneği göster
                data = json.loads(line)
                print(f"\n--- Örnek {i+1} ---")
                print(json.dumps(data, indent=2, ensure_ascii=False))
else:
    print("❌ Veri seti henüz oluşturulmadı")

## 4. YAML Konfigürasyonu ile İleri Seviye Kullanım

Daha fazla kontrol için YAML konfigürasyon dosyası kullanabilirsiniz. Bu yöntemle:
- Prompt'ları özelleştirebilirsiniz
- Farklı parametreler ayarlayabilirsiniz
- Çıktı formatını belirleyebilirsiniz

In [ ]:
# YAML konfigürasyon dosyası oluştur
config_yaml = f"""
# Sistem prompt'u - veri setindeki genel davranışı belirler
dataset_system_prompt: |
  Sen yardımcı bir Türkçe AI asistanısın. Kullanıcılara detaylı ve 
  anlaşılır şekilde yanıt veriyorsun.

# Konu ağacı konfigürasyonu
topic_tree:
  # Ana konu - buradan alt konular türetilecek
  topic_prompt: "Makine Öğrenmesi ve Yapay Zeka"
  
  # LLM provider ayarları
  provider: "{PROVIDER}"
  model: "{LOCAL_MODEL_NAME}"
  temperature: 0.7  # Yaratıcılık seviyesi (0.0-1.0)
  
  # Ağaç yapısı
  degree: 3  # Her konudan kaç alt konu türetilebilir
  depth: 2   # Konu ağacının derinliği

# Veri üretimi ayarları
data_engine:
  # Veri üretim prompt'u - üretilecek verilerin stilini belirler
  generation_system_prompt: |
    Sen bir yapay zeka eğitmenisin. Öğrencilere kavramları açık ve 
    anlaşılır şekilde öğretiyorsun. Her soruya detaylı, örnekli ve 
    adım adım yanıtlar veriyorsun.
  
  temperature: 0.6
  provider: "{PROVIDER}"
  model: "{LOCAL_MODEL_NAME}"

# Veri seti oluşturma parametreleri
dataset:
  creation:
    num_steps: 12      # Toplam kaç veri örneği üretilecek
    batch_size: 2      # Aynı anda kaç örnek üretilecek
    sys_msg: true      # Sistem mesajı dahil edilsin mi
  
  # Çıktı formatı (opsiyonel)
  # builtin://trl_sft_tools  - HuggingFace TRL için
  # builtin://chatml.py      - ChatML formatı
  # builtin://alpaca.py      - Alpaca formatı
  template: "builtin://chatml.py"
"""

# Config dosyasını kaydet
with open('deepfabric_config.yaml', 'w', encoding='utf-8') as f:
    f.write(config_yaml)

print("✓ Konfigürasyon dosyası oluşturuldu: deepfabric_config.yaml")

In [ ]:
# YAML konfigürasyonu ile veri üret
!deepfabric generate \
  --config deepfabric_config.yaml \
  --dataset-save-as dataset_yaml.jsonl

## 5. TXT Dosyasından Soru-Cevap Üretimi

DeepFabric doğrudan txt dosyasından veri üretmez, ancak txt içeriğini **konu (topic)** olarak kullanabilirsiniz.

### Yaklaşım:
1. Txt dosyanızı okuyun
2. İçeriği topic prompt olarak kullanın
3. DeepFabric bu konudan soru-cevaplar üretir

In [ ]:
# Örnek bir txt dosyası oluştur
ornek_metin = """
Python Veri Yapıları

Python'da temel veri yapıları şunlardır:

1. Liste (List): Sıralı, değiştirilebilir koleksiyon. Örnek: [1, 2, 3, 4]
2. Demet (Tuple): Sıralı, değiştirilemez koleksiyon. Örnek: (1, 2, 3)
3. Sözlük (Dictionary): Anahtar-değer çiftleri. Örnek: {'ad': 'Ali', 'yaş': 25}
4. Küme (Set): Sırasız, benzersiz elemanlar. Örnek: {1, 2, 3, 4}

Listeler en çok kullanılan veri yapısıdır çünkü esnektir ve birçok metoda sahiptir.
"""

with open('ornek_konu.txt', 'w', encoding='utf-8') as f:
    f.write(ornek_metin)

print("✓ Örnek txt dosyası oluşturuldu")

In [ ]:
# Txt dosyasını oku ve içeriği topic olarak kullan
with open('ornek_konu.txt', 'r', encoding='utf-8') as f:
    txt_icerik = f.read()

# Özel prompt ile YAML oluştur
txt_config = f"""
dataset_system_prompt: |
  Sen bir Python programlama eğitmenisin. Öğrencilere veri yapılarını 
  açık ve örneklerle öğretiyorsun.

topic_tree:
  # Txt dosyasının içeriğini konu olarak kullanıyoruz
  topic_prompt: |
    {txt_icerik}
    
    Yukarıdaki konular hakkında detaylı alt başlıklar oluştur.
  
  provider: "{PROVIDER}"
  model: "{LOCAL_MODEL_NAME}"
  temperature: 0.7
  degree: 4
  depth: 2

data_engine:
  generation_system_prompt: |
    Python veri yapıları hakkında soru-cevap çiftleri oluştur.
    Her cevap:
    - Açık ve anlaşılır olmalı
    - Kod örnekleri içermeli
    - Pratik kullanım senaryoları göstermeli
  
  temperature: 0.5
  provider: "{PROVIDER}"
  model: "{LOCAL_MODEL_NAME}"

dataset:
  creation:
    num_steps: 16
    batch_size: 2
    sys_msg: true
  template: "builtin://chatml.py"
"""

with open('txt_based_config.yaml', 'w', encoding='utf-8') as f:
    f.write(txt_config)

print("✓ Txt tabanlı konfigürasyon oluşturuldu")

In [ ]:
# Txt içeriğinden soru-cevap üret
!deepfabric generate \
  --config txt_based_config.yaml \
  --dataset-save-as dataset_from_txt.jsonl

In [ ]:
# Üretilen soru-cevapları incele
if os.path.exists('dataset_from_txt.jsonl'):
    print("\n📊 TXT DOSYASINDAN ÜRETİLEN SORU-CEVAPLAR:\n")
    print("=" * 80)
    
    with open('dataset_from_txt.jsonl', 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            if i < 3:  # İlk 3 örneği göster
                data = json.loads(line)
                print(f"\n🔹 Örnek {i+1}:")
                print("-" * 80)
                
                # Messages varsa güzel formatlayalım
                if 'messages' in data:
                    for msg in data['messages']:
                        role = msg.get('role', 'unknown').upper()
                        content = msg.get('content', '')
                        print(f"\n[{role}]")
                        print(content[:500])  # İlk 500 karakter
                        if len(content) > 500:
                            print("... (devamı var)")
                else:
                    print(json.dumps(data, indent=2, ensure_ascii=False)[:1000])
                
                print("\n" + "=" * 80)
else:
    print("❌ Veri seti henüz oluşturulmadı")

## 6. Farklı Veri Formatları

DeepFabric, üretilen veriyi farklı formatlara dönüştürebilir:

In [ ]:
# Mevcut bir veri setini farklı formata çevir
# ChatML formatına çevir
!deepfabric format \
  --dataset dataset_from_txt.jsonl \
  --format builtin://chatml.py \
  --output dataset_chatml.jsonl

print("\n✓ ChatML formatına dönüştürüldü")

In [ ]:
# Alpaca formatına çevir
!deepfabric format \
  --dataset dataset_from_txt.jsonl \
  --format builtin://alpaca.py \
  --output dataset_alpaca.jsonl

print("\n✓ Alpaca formatına dönüştürüldü")

## 7. Parametrelerin Anlamı ve Özelleştirme

### Konu Ağacı Parametreleri:

- **`depth`**: Konu ağacının derinliği. Örnek: `depth=3` → Ana konu → Alt konu → Alt-alt konu
- **`degree`**: Her konudan kaç alt konu üretilir. Örnek: `degree=4` → Her konudan 4 alt konu
- **`topic_prompt`**: Ana konu. Buradan tüm alt konular türetilir

**Toplam konu sayısı**: `degree^depth` (örn: 3^2 = 9 konu)

### Veri Üretim Parametreleri:

- **`num_steps`**: Toplam kaç veri örneği üretilecek
- **`batch_size`**: Aynı anda kaç örnek üretilecek (paralel işlem)
- **`temperature`**: Yaratıcılık seviyesi (0.0 = deterministik, 1.0 = yaratıcı)
- **`sys_msg`**: Sistem mesajı dahil edilsin mi?

### Prompt'lar:

- **`dataset_system_prompt`**: Genel sistem davranışı
- **`generation_system_prompt`**: Veri üretim stili
- **`topic_prompt`**: Ana konu (txt dosyanızın içeriği de olabilir)

## 8. Özel Senaryolar

### Senaryo 1: Kod Örnekleri Üretimi

In [ ]:
kod_config = f"""
dataset_system_prompt: |
  Sen bir programlama eğitmenisin. Her yanıtta çalışan kod örnekleri veriyorsun.

topic_tree:
  topic_prompt: |
    Python ile veri analizi: Pandas, NumPy ve Matplotlib kullanımı
  
  provider: "{PROVIDER}"
  model: "{LOCAL_MODEL_NAME}"
  temperature: 0.6
  degree: 3
  depth: 2

data_engine:
  generation_system_prompt: |
    Veri analizi hakkında soru-cevap üret. Her cevap:
    1. Konsepti açıkla
    2. Kod örneği ver (çalışır kod)
    3. Çıktıyı göster
    4. Kullanım alanlarını açıkla
  
  temperature: 0.4
  provider: "{PROVIDER}"
  model: "{LOCAL_MODEL_NAME}"

dataset:
  creation:
    num_steps: 20
    batch_size: 2
    sys_msg: true
"""

with open('kod_ornekleri_config.yaml', 'w', encoding='utf-8') as f:
    f.write(kod_config)

print("✓ Kod örnekleri konfigürasyonu hazır")

### Senaryo 2: Çok Turlu Konuşmalar

In [ ]:
konusma_config = f"""
dataset_system_prompt: |
  Sen yardımcı bir müşteri hizmetleri temsilcisisin. Nazik ve çözüm odaklısın.

topic_tree:
  topic_prompt: |
    E-ticaret müşteri hizmetleri: Sipariş sorunları, iade işlemleri, 
    ürün bilgileri, teslimat takibi
  
  provider: "{PROVIDER}"
  model: "{LOCAL_MODEL_NAME}"
  temperature: 0.7
  degree: 4
  depth: 2

data_engine:
  generation_system_prompt: |
    Müşteri-temsilci konuşmaları oluştur. Her konuşma:
    - Gerçekçi olmalı
    - 3-5 mesaj alışverişi içermeli
    - Sorunu çözmeli
    - Profesyonel ama samimi olmalı
  
  temperature: 0.7
  provider: "{PROVIDER}"
  model: "{LOCAL_MODEL_NAME}"

dataset:
  creation:
    num_steps: 15
    batch_size: 1
    sys_msg: true
  template: "builtin://chatml.py"
"""

with open('konusma_config.yaml', 'w', encoding='utf-8') as f:
    f.write(konusma_config)

print("✓ Konuşma konfigürasyonu hazır")

## 9. Veri Setini İnceleme ve İstatistikler

In [ ]:
def veri_seti_analizi(dosya_yolu):
    """
    JSONL formatındaki veri setini analiz eder
    """
    if not os.path.exists(dosya_yolu):
        print(f"❌ Dosya bulunamadı: {dosya_yolu}")
        return
    
    veriler = []
    with open(dosya_yolu, 'r', encoding='utf-8') as f:
        for line in f:
            veriler.append(json.loads(line))
    
    print(f"\n📊 Veri Seti Analizi: {dosya_yolu}")
    print("=" * 80)
    print(f"Toplam örnek sayısı: {len(veriler)}")
    
    # Mesaj sayıları
    if veriler and 'messages' in veriler[0]:
        mesaj_sayilari = [len(v['messages']) for v in veriler if 'messages' in v]
        print(f"Ortalama mesaj sayısı: {sum(mesaj_sayilari) / len(mesaj_sayilari):.1f}")
        print(f"Min mesaj sayısı: {min(mesaj_sayilari)}")
        print(f"Max mesaj sayısı: {max(mesaj_sayilari)}")
    
    # Alan isimleri
    if veriler:
        print(f"\nVeri alanları: {list(veriler[0].keys())}")
    
    # İlk örneği göster
    if veriler:
        print("\n📝 Örnek veri:")
        print("-" * 80)
        print(json.dumps(veriler[0], indent=2, ensure_ascii=False)[:1500])
        print("...")
    
    return veriler

# Örnek kullanım
# veri_seti_analizi('dataset_from_txt.jsonl')

## 10. Model Eğitimi için Veri Hazırlığı

Üretilen veriyi model eğitimi için kullanmak:

In [ ]:
# HuggingFace format için
!deepfabric format \
  --dataset dataset_from_txt.jsonl \
  --format builtin://trl_sft_tools \
  --output dataset_hf_ready.jsonl

print("\n✓ HuggingFace TRL için hazır veri seti oluşturuldu")
print("\nBu veri setini şu şekilde kullanabilirsiniz:")
print("""
from datasets import load_dataset
from trl import SFTTrainer

dataset = load_dataset('json', data_files='dataset_hf_ready.jsonl')
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset['train'],
    ...
)
trainer.train()
""")

## 11. İpuçları ve En İyi Pratikler

### 🎯 Kaliteli Veri Üretimi İçin:

1. **Spesifik Konular Seçin**: 
   - ❌ "Programlama" yerine
   - ✅ "Python'da list comprehension kullanımı ve performans optimizasyonu"

2. **Prompt'ları Özelleştirin**:
   - `generation_system_prompt` ile veri stilini belirleyin
   - Örnekler, açıklamalar, kod blokları gibi isteklerinizi ekleyin

3. **Temperature Ayarları**:
   - Konu oluşturma: 0.7-0.8 (daha yaratıcı)
   - Veri üretimi: 0.4-0.6 (daha tutarlı)
   - Teknik içerik: 0.3-0.5 (daha deterministik)

4. **Küçük Başlayın**:
   - İlk denemede `num_steps=5-10` kullanın
   - Kaliteyi kontrol edin
   - Sonra artırın

5. **Batch Size**:
   - Lokal API: 1-2 (kaynak sınırlı)
   - Güçlü API: 4-8 (daha hızlı)

### 🔧 Sorun Giderme:

**API Bağlantı Hatası:**
```python
# API'nizin çalıştığından emin olun
import requests
response = requests.get(os.environ['OPENAI_BASE_URL'].replace('/v1', '/health'))
print(response.status_code)  # 200 olmalı
```

**Düşük Kaliteli Veri:**
- `generation_system_prompt` daha detaylı yapın
- Temperature'ı düşürün
- Daha güçlü bir model deneyin

**Yavaş Üretim:**
- `batch_size` artırın
- Daha küçük bir model kullanın
- `depth` ve `degree` azaltın

## 12. Özet ve Sonraki Adımlar

### 📝 Öğrendikleriniz:

✅ DeepFabric'i kurdunuz ve yapılandırdınız  
✅ Lokal LLM API ile bağlandınız  
✅ Basit ve gelişmiş veri setleri oluşturdunuz  
✅ Txt dosyasından soru-cevap ürettiniz  
✅ Farklı formatları öğrendiniz  
✅ Prompt'ları özelleştirdiniz  

### 🚀 Sonraki Adımlar:

1. **Kendi Verilerinizi Oluşturun**: 
   - Kendi txt dosyalarınızı kullanın
   - Domain-spesifik konular seçin

2. **Kaliteyi Artırın**:
   - Farklı prompt'lar deneyin
   - Temperature ayarlarını optimize edin
   - Üretilen veriyi gözden geçirin

3. **Model Eğitimi**:
   - Ürettiğiniz veri ile modelinizi fine-tune edin
   - QLoRA veya LoRA kullanın
   - Sonuçları değerlendirin

4. **İleri Seviye**:
   - Özel formatlar yazın
   - Araç çağrıları (tool calling) ekleyin
   - Multi-turn konuşmalar oluşturun

### 📚 Kaynaklar:

- GitHub: https://github.com/lukehinds/deepfabric
- Dokümantasyon: Repository README
- Örnekler: `examples/` klasörü

---

**İyi çalışmalar! 🎉**